In [16]:
from dotenv import load_dotenv
import os
import requests
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np

In [17]:
load_dotenv('api.env')

True

In [18]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [19]:
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [20]:
alpaca = tradeapi.REST(
    alpaca_api_key,alpaca_secret_key,
    api_version="v2")

In [21]:
symbol='SPY'
timeframe = '1H'
start_date = pd.Timestamp("2013-01-01", tz="America/New_York").isoformat()

In [22]:
historical_data = alpaca.get_bars(symbol, timeframe, start=start_date).df
historical_data['delta']=historical_data['close'].diff()

In [23]:
historical_data

,open,high,low,close,volume,trade_count,vwap,delta
timestamp,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.44,209.24,209.3000,30375,82,209.322606,NaN
2015-12-01 10:00:00+00:00,209.30,209.36,209.07,209.3600,54490,118,209.214253,0.0600
2015-12-01 11:00:00+00:00,209.37,209.65,209.37,209.4400,44404,109,209.526441,0.0800
2015-12-01 12:00:00+00:00,209.45,209.59,209.33,209.5100,111106,234,209.460429,0.0700
2015-12-01 13:00:00+00:00,209.52,209.67,209.27,209.3200,426233,645,209.429479,-0.1900
...,...,...,...,...,...,...,...,...
2023-03-24 19:00:00+00:00,394.67,395.84,390.09,395.7200,19815554,151383,394.974732,1.0700
2023-03-24 20:00:00+00:00,395.71,396.55,395.50,396.5000,12658941,9699,395.531286,0.7800
2023-03-24 21:00:00+00:00,396.49,396.53,395.75,396.4400,565192,852,395.816342,-0.0600


**VWAP Trading Signal**

We decide to buy/sell at points where the market price crosses the vwap.

In [24]:
historical_data['Predicted']=np.where(historical_data['vwap']>=historical_data['close'], 1,0)
historical_data['entry/exit']=historical_data['Predicted'].diff()
historical_data


,open,high,low,close,volume,trade_count,vwap,delta,Predicted,entry/exit
timestamp,,,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.44,209.24,209.3000,30375,82,209.322606,NaN,1,NaN
2015-12-01 10:00:00+00:00,209.30,209.36,209.07,209.3600,54490,118,209.214253,0.0600,0,-1.0
2015-12-01 11:00:00+00:00,209.37,209.65,209.37,209.4400,44404,109,209.526441,0.0800,1,1.0
2015-12-01 12:00:00+00:00,209.45,209.59,209.33,209.5100,111106,234,209.460429,0.0700,0,-1.0
2015-12-01 13:00:00+00:00,209.52,209.67,209.27,209.3200,426233,645,209.429479,-0.1900,1,1.0
...,...,...,...,...,...,...,...,...,...,...
2023-03-24 19:00:00+00:00,394.67,395.84,390.09,395.7200,19815554,151383,394.974732,1.0700,0,0.0
2023-03-24 20:00:00+00:00,395.71,396.55,395.50,396.5000,12658941,9699,395.531286,0.7800,0,0.0
2023-03-24 21:00:00+00:00,396.49,396.53,395.75,396.4400,565192,852,395.816342,-0.0600,0,0.0


In [25]:
up_mean=historical_data.loc[:,['delta']]


In [26]:
up_mean=historical_data.loc[historical_data['delta'] > 0,'delta'].mean(skipna=True)
down_mean=abs(historical_data.loc[historical_data['delta'] < 0,'delta'].mean(skipna=True))
rs=up_mean/down_mean
rs
#100-(100/(1+rs))
#period=historical_data.loc[:14,]
#period
                        

0.931382976856071

In [27]:
mean_positive = np.where(historical_data['delta']> 0, historical_data['delta']).mean()
mean_positive

ValueError: either both or neither of x and y should be given

In [28]:
from datetime import datetime, timedelta
n_days_ago = datetime.now() - timedelta(days=7)
period=historical_data.loc[n_days_ago:,'delta']
period
start_time = pd.to_datetime('2023-03-20 08:00:00+00:00')

/var/folders/49/gpk50n7s6nj72g6n3hktw_tm0000gn/T/ipykernel_2241/1369228583.py:3: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  period=historical_data.loc[n_days_ago:,'delta']


**RSI**

The stock is considered overbought when the RSI rises above 70 and considered oversold when the RSI dips below 30 

In [35]:
import numpy as np

def calculate_rsi(price_diff, n=14):
    rsi=[0]*len(price_diff)
    #first n days
    up_mean=price_diff.head(n)[price_diff>0].mean(skipna=True)
    down_mean=abs(price_diff.head(n)[price_diff<=0].mean(skipna=True))
    rs=float(up_mean/down_mean)
    rsi[:n]=[100-(100/(1+rs))]*n

    for i in range(n,len(price_diff)):
        up_mean=price_diff[i-n:i][price_diff>0].mean()
        down_mean=abs(price_diff[i-n:i][price_diff<=0]).mean(skipna=True)
        rs=float(up_mean/down_mean)
        rsi[i]=100-(100/(1+rs))
    return rsi
historical_data['rsi'] = calculate_rsi(historical_data.loc[:,['delta']])




In [36]:
historical_data

,open,high,low,close,volume,trade_count,vwap,delta,Predicted,entry/exit,rsi
timestamp,,,,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.44,209.24,209.3000,30375,82,209.322606,NaN,1,NaN,64.376590
2015-12-01 10:00:00+00:00,209.30,209.36,209.07,209.3600,54490,118,209.214253,0.0600,0,-1.0,64.376590
2015-12-01 11:00:00+00:00,209.37,209.65,209.37,209.4400,44404,109,209.526441,0.0800,1,1.0,64.376590
2015-12-01 12:00:00+00:00,209.45,209.59,209.33,209.5100,111106,234,209.460429,0.0700,0,-1.0,64.376590
2015-12-01 13:00:00+00:00,209.52,209.67,209.27,209.3200,426233,645,209.429479,-0.1900,1,1.0,64.376590
...,...,...,...,...,...,...,...,...,...,...,...
2023-03-24 19:00:00+00:00,394.67,395.84,390.09,395.7200,19815554,151383,394.974732,1.0700,0,0.0,43.724526
2023-03-24 20:00:00+00:00,395.71,396.55,395.50,396.5000,12658941,9699,395.531286,0.7800,0,0.0,45.603675
2023-03-24 21:00:00+00:00,396.49,396.53,395.75,396.4400,565192,852,395.816342,-0.0600,0,0.0,47.442942


In [41]:
from finta import TA

In [49]:
ema3 = TA.EMA(historical_data, period=3)
ema5 = TA.EMA(historical_data, period=5)
ema8 = TA.EMA(historical_data,period=8)
ema13 = TA.EMA(historical_data,period=13)

historical_data['ema-3']=ema3
historical_data['ema-5']=ema5
historical_data['ema-8']=ema8
historical_data['ema-13']=ema13



In [50]:
historical_data

,open,high,low,close,volume,trade_count,vwap,delta,Predicted,entry/exit,rsi,ema-3,ema-5,ema-8,ema-13
timestamp,,,,,,,,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.44,209.24,209.3000,30375,82,209.322606,NaN,1,NaN,64.376590,209.300000,209.300000,209.300000,209.300000
2015-12-01 10:00:00+00:00,209.30,209.36,209.07,209.3600,54490,118,209.214253,0.0600,0,-1.0,64.376590,209.340000,209.336000,209.333750,209.332308
2015-12-01 11:00:00+00:00,209.37,209.65,209.37,209.4400,44404,109,209.526441,0.0800,1,1.0,64.376590,209.397143,209.385263,209.378342,209.373858
2015-12-01 12:00:00+00:00,209.45,209.59,209.33,209.5100,111106,234,209.460429,0.0700,0,-1.0,64.376590,209.457333,209.437077,209.424486,209.416118
2015-12-01 13:00:00+00:00,209.52,209.67,209.27,209.3200,426233,645,209.429479,-0.1900,1,1.0,64.376590,209.386452,209.392133,209.392028,209.390564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-24 19:00:00+00:00,394.67,395.84,390.09,395.7200,19815554,151383,394.974732,1.0700,0,0.0,43.724526,394.815240,394.184390,393.631230,393.339405
2023-03-24 20:00:00+00:00,395.71,396.55,395.50,396.5000,12658941,9699,395.531286,0.7800,0,0.0,45.603675,395.657620,394.956260,394.268734,393.790918
2023-03-24 21:00:00+00:00,396.49,396.53,395.75,396.4400,565192,852,395.816342,-0.0600,0,0.0,47.442942,396.048810,395.450840,394.751238,394.169358
